# Final NN course CNN project with resnet (part 2)

🧑‍🎓 students :

- Ali Fadaeimanesh (40311422011)
- Ali Ebadi (40311422016)
- Fatemeh Tahaei (40211403004)
- Armin Ahangar (40311422014)
- Farbod SeyedAli ()


In [1]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("usefashrfi/iran-used-cars-dataset")

print("Path to dataset files:", path)

Path to dataset files: /teamspace/studios/this_studio/.cache/kagglehub/datasets/usefashrfi/iran-used-cars-dataset/versions/1


In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time

# initial settings
BASE_DIR = "/teamspace/studios/this_studio/.cache/kagglehub/datasets/usefashrfi/iran-used-cars-dataset/versions/1/iran-used-cars-dataset/"
SPLIT_DIR = os.path.join(BASE_DIR, "split")
TRAIN_DIR = os.path.join(SPLIT_DIR, "train")
VAL_DIR = os.path.join(SPLIT_DIR, "val")
TEST_DIR = os.path.join(SPLIT_DIR, "test")  # در صورت نیاز

# check gpu access
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# data transform settings
data_transforms = {
    "train": transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize(
                [
                    0.485,
                    0.456,
                    0.406,
                ],  # مقادیر نرمال‌سازی استاندارد برای مدل‌های پیش‌آموزش‌دیده
                [0.229, 0.224, 0.225],
            ),
        ]
    ),
    "val": transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
    "test": transforms.Compose(
        [  # در صورت نیاز
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

#  checking for dirs existance
for directory in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    if not os.path.isdir(directory):
        print(f"Warning: Directory {directory} does not exist.")

# loading data using imagefolder
image_datasets = {
    "train": datasets.ImageFolder(TRAIN_DIR, transform=data_transforms["train"]),
    "val": datasets.ImageFolder(VAL_DIR, transform=data_transforms["val"]),
    "test": datasets.ImageFolder(TEST_DIR, transform=data_transforms["test"]),
}

# dataloader setup
batch_size = 32
num_workers = 4

dataloaders = {
    "train": DataLoader(
        image_datasets["train"],
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    ),
    "val": DataLoader(
        image_datasets["val"],
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    ),
    "test": DataLoader(
        image_datasets["test"],
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    ),  # در صورت نیاز
}

dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "val", "test"]}
class_names = image_datasets["train"].classes
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")
print(f"Training samples: {dataset_sizes['train']}")
print(f"Validation samples: {dataset_sizes['val']}")

# resnet model define
model = models.resnet18(pretrained=True)

# configure the output layer sizes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

model = model.to(device)

# error function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# learning plan
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


# train and evaluation
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        # Each epoch consists of training and validation phases
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()  # Set the model to training mode
            else:
                model.eval()  # Set the model to evaluation mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over the data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the gradients
                optimizer.zero_grad()

                # Forward pass only in training phase
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimization only in training phase
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == "train":
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Save the best model
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best val Acc: {best_acc:.4f}")

    # Load the best weights
    model.load_state_dict(best_model_wts)
    return model


# Train the model
num_epochs = 25
model = train_model(model, criterion, optimizer, scheduler, num_epochs=num_epochs)

# Save the trained model
model_path = "best_resnet_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Load model from file
# MODEL_PATH = './best_resnet_model.pth'
# model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
# model.eval()  # Set the model to evaluation mode
# print("Model successfully loaded.")


def evaluate_model(model, dataloader, device):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloaders["val"]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            running_corrects += torch.sum(preds == labels.data)

    total_acc = running_corrects.double() / dataset_sizes["val"]
    print(f"Validation Accuracy: {total_acc:.4f}")


evaluate_model(model, dataloaders["val"], device)


# (Optional) Evaluation on the test set
def evaluate_test(model, dataloader, device):
    model.eval()
    running_corrects = 0
    dataset_size = len(dataloader.dataset)

    with torch.no_grad():
        for inputs, labels in dataloaders["test"]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            running_corrects += torch.sum(preds == labels.data)

    total_acc = running_corrects.double() / dataset_size
    print(f"Test Accuracy: {total_acc:.4f}")


evaluate_test(model, dataloaders["test"], device)

Using device: cuda


Number of classes: 13
Training samples: 6809
Validation samples: 1454


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/25
----------
train Loss: 0.7574 Acc: 0.7547
val Loss: 0.6944 Acc: 0.7889

Epoch 2/25
----------
train Loss: 0.3640 Acc: 0.8853
val Loss: 0.3740 Acc: 0.8851

Epoch 3/25
----------
train Loss: 0.2383 Acc: 0.9248
val Loss: 0.2768 Acc: 0.9257

Epoch 4/25
----------
train Loss: 0.1892 Acc: 0.9366
val Loss: 0.4056 Acc: 0.8755

Epoch 5/25
----------
train Loss: 0.1882 Acc: 0.9430
val Loss: 0.2292 Acc: 0.9264

Epoch 6/25
----------
train Loss: 0.1490 Acc: 0.9529
val Loss: 0.2180 Acc: 0.9312

Epoch 7/25
----------
train Loss: 0.1158 Acc: 0.9648
val Loss: 0.2735 Acc: 0.9154

Epoch 8/25
----------
train Loss: 0.0453 Acc: 0.9865
val Loss: 0.0937 Acc: 0.9759

Epoch 9/25
----------
train Loss: 0.0285 Acc: 0.9918
val Loss: 0.0824 Acc: 0.9787

Epoch 10/25
----------
train Loss: 0.0162 Acc: 0.9954
val Loss: 0.0770 Acc: 0.9773

Epoch 11/25
----------
train Loss: 0.0147 Acc: 0.9969
val Loss: 0.0800 Acc: 0.9794

Epoch 12/25
----------
train Loss: 0.0107 Acc: 0.9978
val Loss: 0.0782 Acc: 0.9787

E